In [3]:
import pandas as pd
customers = pd.read_csv('D:/personalized_offers_project/data/customers.csv')
transactions = pd.read_csv('D:/personalized_offers_project/data/offers.csv')
offers = pd.read_csv('D:/personalized_offers_project/data/redemptions.csv')
redemptions = pd.read_csv('D:/personalized_offers_project/data/transactions.csv')

In [ ]:
import numpy as np
now = pd.to_datetime('2025-10-05')
rfm = transactions.groupby('customer_id').agg(
    recency=('timestamp', lambda x: (now - pd.to_datetime(x).max()).days),
    frequency=('order_id','count'),
    monetary=('total_amount','sum')
).reset_index()

KeyError: 'customer_Id'

In [ ]:
data = rfm.merge(customers, on='customer_id', how='left')
red_rate = redemptions.groupby('customer_id')['offer_id'].count()/transactions.groupby('customer_id')['order_id'].count()
data['redemption_rate'] = data['customer_id'].map(red_rate).fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
X = data[['recency','frequency','monetary']].values
X_scaled = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=5, random_state=42).fit(X_scaled)
data['segment'] = kmeans.labels_

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Create target (1 if customer redeemed any offer)
target = (redemptions.groupby('customer_id')['offer_id'].count()>0).astype(int)
data['target'] = data['customer_id'].map(target).fillna(0)

features = ['recency','frequency','monetary','redemption_rate']
X_train, X_val, y_train, y_val = train_test_split(data[features], data['target'], test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
clf.fit(X_train, y_train)
data['pred_prob'] = clf.predict_proba(data[features])[:,1]

In [ ]:
N = len(data)
uplift = 0.05  # 5% incremental conversion
AOV = 8.0
margin = 0.6
discount_cost = 1.5
fixed_costs = 2000
incremental_redemptions = N*uplift
incremental_revenue = incremental_redemptions * AOV
incremental_profit = incremental_revenue * margin
total_costs = incremental_redemptions * discount_cost + fixed_costs
net_profit = incremental_profit - total_costs
roi = net_profit / total_costs

print(f"Simulated Net Profit: ${net_profit:,.2f}, ROI: {roi*100:.2f}%")